In [1]:
import numpy as np
import matplotlib.pylab as plt
from pylab import *
import sys
import pandas as pd

In [2]:
##########################
# define a few constants #		##fft python random phase
##########################

e0 = np.loadtxt('837nm.dat')		#load the reference pulse, pump @836 nm
e0 = e0.T
e1 = np.sqrt(e0[1])
e1_ph = e0[2]
e = np.array(e1)*np.exp(-1j*(np.array(e1_ph)))		#reforming E-field

original_trace = np.loadtxt('trace/trace1.frg')		#load FROG trace from experiment

N = len(e0[0])		#timesteps
N_max = np.max(e0[0])
N_min = np.min(e0[0])

t = e0[0]-N_min
T0 = N_max-N_min	#total time duration
dt = T0/(N-1)
df = 1.0/T0		#frequency steps
TD = N			#time delay for FROG-trace generation
freq = np.arange(0,1/dt+df,df)*1000

In [3]:
original_trace.shape

(128, 128)

In [4]:
## for generation of Gaussian pulses
def gaussian_pulse(T,f,A,phase,tau):
	w0 = f*2*np.pi		#central angular frequency of pulse
	temp = -1j*w0*(t-tau) + 1j*phase
	E = A*np.sqrt(np.exp(-4.0*np.log(2.0)/np.log(np.e)*((t-tau)/T)**2))*np.exp(-temp)
	return E

In [5]:
## calculate cross correlation of two pulses
def xcorr(e0,e,td):		#shift one pulse by time delay of 'td'
	E_delay = e*1.0
	if td<=0:
		E_delay[0:N+td] = e0[-td:]
		E_delay[N+td:] = [0]*(-td)
	else:
		E_delay[0:td] = [0]*(td)
		E_delay[td:] = e0[0:N-td]	
	return  e*E_delay

In [6]:
## calculate the differert of trace A and trace B
def array_diff(A,B):
	A = A/np.max(A)
	B = B/np.max(B)
	return np.sum(np.square(A-B))/np.sum(np.square(A))

In [7]:
def FROG_generator(e,E,TD):	
	FROG = np.zeros((TD,N),dtype=float)
	for td in range(-TD//2,TD//2):
		E_corre = xcorr(e,E,td)
		temp0 = np.fft.fftshift(np.fft.fft(E_corre))
		FROG[(td+TD//2)-1] = abs(temp0)**2	
	return FROG.T

In [8]:
## plot out FROG trace
def FROG_trace_plot(FROG):
	fig,ax = plt.subplots()
	plt.imshow(FROG)
	plt.title("FROG trace")
	plt.grid(True)
	plt.show()

In [9]:
## calculate FROG traces with different time delays
def FROGs(T1,T2,A1,A2,f0,f1,tau1):
    tau2 = tau1-700
    phase1 = a2*(t-tau1)**2+a3*(t-tau1)**3
    phase2 = a2*(t-tau2)**2+a3*(t-tau2)**3 
    E1 = gaussian_pulse(T1,f0,A1,phase1,tau1)
    E2 = gaussian_pulse(T2,f1,A2,phase2,tau2)
    E = E1+E2
    FROG = FROG_generator(e,E,TD)
    delta = array_diff(FROG, original_trace)	
    return FROG, delta

In [11]:
######################
# initial parameters #
######################

T1 = np.arange(295,695)
#T1 =                                    #Gaussian pulse duration in fs, FW
T2 = np.arange(300,1000)
#T2 = 287.7733748374851
A1 = np.arange(0.6,1.5,0.05)
#A1 = 0.7524039393439459                #Amplitude of Raman pulse
A2 = np.arange(0.6,1.5,0.05)
#A2 = 1.369630361735096

a2 = 2.50*10**-6
#a2 = np.delete(a2, np.where(np.abs(a2) < 1e-10))#second order phase factor
a3 = 3.0*10**-9		#third order phase factor
#a3 = np.delete(a3, np.where(np.abs(a3) < 1e-14))

F0 = np.arange(-0.005, 0.005, 0.0005)
F0 = np.delete(F0, np.where(np.abs(F0) < 1e-10))    #Raman center-wavelength
#f0 = 0.0014144712047722094
F1 = np.arange(-0.0005, 0.0005, 0.00005)
F1 = np.delete(F1, np.where(np.abs(F1) < 1e-10))    #Red-shifted center-wavelength
#f1 = 0.0002521943119991874
			
i0 = 333.0		#unit: fs
Tau1 = np.arange(-1000, 5000, 50)	#Raman pulse peak time
#tau1 = 3325.2480418676446
#tau2 = tau1-700		#Red-shifted pulse peak time

In [14]:
F1

array([-5.0e-04, -4.5e-04, -4.0e-04, -3.5e-04, -3.0e-04, -2.5e-04,
       -2.0e-04, -1.5e-04, -1.0e-04, -5.0e-05,  5.0e-05,  1.0e-04,
        1.5e-04,  2.0e-04,  2.5e-04,  3.0e-04,  3.5e-04,  4.0e-04,
        4.5e-04])

In [15]:
values = pd.DataFrame(columns = ['T1', 'T2', 'A1', 'A2', 'F0', 'F1', 'Tau1'])
values

,T1,T2,A1,A2,F0,F1,Tau1


In [18]:
###########################
# FROG trace main program #
###########################
FROGS = []
values = []
for i in range(1000):
    t1 = np.random.choice(T1)
    t2 = np.random.choice(T2)
    ai1 = np.random.choice(A1)
    ai2 = np.random.choice(A2)
    f0 = np.random.choice(F0)
    f1 = np.random.choice(F1)
    #aa2 = np.random.choice(a2)
    #aa3 = np.random.choice(a3)
    tau1 = np.random.choice(Tau1)
    (FROG, delta) = FROGs(t1,t2,ai1,ai2,f0,f1,tau1)
    print(delta)
    values.append([t1, t2, ai1, ai2, f0, f1, tau1])
    FROGS.append(FROG)

1.0835416824964168
2.1485588790568344
2.714109915183668
2.0487897594356244
2.36847059409164
1.7310312524467086
1.3242774252171032
2.6016628759604195
1.87351464056625
2.914805474047576
1.453791765738137
2.3799965321508014
2.9305680129932874
2.4922209158873807
2.203295261768355
1.7465163704535
2.5642281888286282
3.0819048897123014
2.507944902199012
1.838720606233159
1.9987757595233766
3.0124798446156764
1.7773667129806734
2.3994016929113786
1.782367981072131
2.005260070038331
1.7533628485029191
0.885084195350434
2.294102741857631
2.831438713108144
2.114771123984366
2.739728896957978
1.5862164536243084
2.9467193824123554
2.446880514871441
1.6879828253917288
1.9514670415228998
0.4640542059456962
2.206586768104541
2.5295485127868433
1.9996094221546825
2.075103690309829
0.8801089329353085
2.8765769370317855
1.0899409557845485
2.8653058797773365
2.4638415063888
2.927944012518755
1.4148862849566244
1.7193801436726457
1.398504230477874
2.3448293571696226
1.6001977388059778
1.5370104876836164
2.

2.761155941200064
2.639514498821849
2.03940383749768
1.7621107679315495
2.3229305248035783
0.4856437298045555
2.115254388512656
2.93409108071381
0.7670769422553119
2.1214637562674645
2.404724180255214
2.807049901633305
2.2756787682753545
1.3299782610049384
2.648752272143993
2.5370170199705773
2.94286023826849
2.4292559572564874
1.999621215300082
2.7765377263152744
1.1524462972180516
1.1813637423402448
2.4564921103669253
1.9091529212703746
2.6179085303687355
2.2143791595252558
1.4031088529235256
1.7261531791318978
2.428760874325156
1.8325044027301698
2.5991646244817583
1.4120355517593717
2.2018482612477857
2.9035699599205516
2.8071834970481735
1.8752625898635091
2.7917580206337766
2.395550864283406
1.382888959120049
2.0315515937581194
2.1168939420497743
1.5627353046003176
2.6544652951042798
2.8451690825134297
3.0105611460723183
2.22788077312495
1.9867375780450853
1.9607716079560535
2.492035385339052
2.4180787299133586
1.9576091518328855
2.195789156484035
2.19930491335454
2.9420124457261

2.1750529892855455
2.5073602743867056
2.942476249245027
3.0056721807576703
2.459768475840346
1.6335868948889096
1.4432692334909158
2.20989798583648
2.242958296740888
2.013199976869383
2.533921385174168
2.722943595550444
1.7356276492664005
1.962800756242593
2.6290855477654564
1.8584288079602989
2.653103161105364
2.493554540428265
2.606249676527281
2.328444222586742
1.911584672277099
1.9035322140901743
2.251614015542766
0.725179449865174
2.9731222356739755
1.3411200917016466
2.808888431545977
1.2915243599906516
1.5280479773748672
2.404639278406303
2.462581484154627
1.5649227058389639
2.3425438902426805
2.690596664176559
2.86044190611131
2.2486227659072755
1.4907861243636653
2.501718029031072
2.346094036848776
2.2140481003070183
1.774917373331433
1.2023432352365753
1.4882684550743737
1.1321848704552988
1.6315278322689741
1.7935886983194287
2.895955743100926
2.5689481144566213
2.369772967007668
1.5538609242317774
2.356088790020676
2.101431917553771
1.145744527259317
2.727508034247036
2.697

In [19]:
FROGS = np.array(FROGS)
#FROGS = FROGS.reshape(1000,-1)

In [20]:
values = np.array(values)
values.shape

(1000, 7)

In [21]:
np.save("FROG_1k", FROGS)

In [22]:
np.save("labels_ann_1k", values)